**Title**

Using R and knn regression to predict used car costs based on mileage and year

**Introduction**

Being able to predict used car prices can save families thousands of dollars. By using the progam language R, we can plot graphs and get a prediction for how much a car may cost. Using a method called regression, we can predict a numerical value based on other influential factors. For example, we might be able to predict that newer cars tend to cost more than old used cars! We will predict the cost of a used car based on factors like mileage and the year of a car. To do this, we will use a dataset pulled form the web in a csv format called USA_cars_datasets.csv. 

**Preliminary exploratory analysis**

We can simply read the online file my setting our url to the specified url then using read_csv() of the url. For example, 

In [19]:
library(tidyverse)
library(rvest)

url <- "https://www.kaggle.com/datasets/goyalshalini93/car-data/CarPrice_Assignment.csv"
car <- read.csv(url)
car


Warning message in file(file, "rt"):
“cannot open URL 'https://www.kaggle.com/datasets/goyalshalini93/car-data/CarPrice_Assignment.csv': HTTP status was '404 Not Found'”


ERROR: Error in file(file, "rt"): cannot open the connection to 'https://www.kaggle.com/datasets/goyalshalini93/car-data/CarPrice_Assignment.csv'


**Methods**

**Expected Outcomes and significance**
